In [1]:
%run ../../modulos.ipynb

In [47]:
def request_lichess(url):    
    headers = {"Authorization": f"Bearer {personal_access_token}"}    
    response = requests.get(url, headers=headers)    
    return response

def obtener_estadisticas(usuario, perf="blitz"):
    data = requests.get(f'https://lichess.org/api/user/{usuario}/perf/{perf}')
    return data.json()     

## Conexión a Mongo

~~~
docker run --name contenedor-mongo -p 27017:27017 mongo
~~~

In [2]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
db_Lichess=client['Lichess'] # database
equipos_collection=db_Lichess['equipos'] # colección
usuarios_col=db_Lichess['usuarios'] # colección

# Conexión a Neo4j

`docker run --name "Lichess_neo" -p 7474:7474 -p 7687:7687 neo4j `

In [3]:
from neo4j import GraphDatabase

data_base_connection = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "1234"))
session = data_base_connection.session() 

## Obtencion equipos liches API

In [24]:
grupos_unam_url = 'https://lichess.org/api/team/search?text=UNAM'
data = request_lichess(grupos_unam_url).json()

n_equipos = data['maxPerPage']
equipos = data['currentPageResults']
# Inserción de equipos en la base de datos
for i in range(n_equipos):
    equipos_collection.insert_one(equipos[i]).inserted_id

## Obtención de usuarios Lichess API

Obtendremos 10 jugadores por cada equipo a traves de la API.

In [66]:
from tqdm.notebook import tqdm_notebook

for equipo in equipos:
    usuario_raw = requests.get(f'https://lichess.org/api/team/{equipo["id"]}/users')
    usuario_raw = usuario_raw.text.split('\n')[:-1]
    # Tomamos maximos 10 usuarios
    usuarios = list(map(json.loads, usuario_raw))
    # Guardarlo en mongo    
    for usuario in tqdm_notebook(usuarios[:10]):
        id_usuario = usuario['id']
        instruccion = f'''
      MERGE (u:Usuario {{id:"{id_usuario}"}})
      '''
        usuarios_col.insert_one({id_usuario: usuario})
        try:
            est_usuario = obtener_estadisticas(id_usuario)
        except:
            break
        usuarios_col.update_one({id_usuario: {'$exists': 'true'}}, {
                                '$set': est_usuario})

        # oponentes
        oponentes_fuertes = est_usuario['stat']['bestWins']['results'][:]
        ids_oponentes = [oponente['opId']['id']
                         for oponente in oponentes_fuertes]

        j = 0
        i = 0
        while i < len(oponentes_fuertes) and j < 3:
            # for id_op in ids_oponentes:

            id_op = ids_oponentes[i]

            i = i+1
            try:
                est_usuario1 = obtener_estadisticas(id_op)
            except:
                continue
            j = j+1
            usuarios_col.insert_one({id_op: ''})
            usuarios_col.update_one({id_op: {'$exists': 'true'}}, {
                                    '$set': est_usuario1})
            instruccion += f'''
        MERGE (u{i}:Usuario {{id:"{id_op}"}})
        MERGE (u)-[:gano_a]->(u{i})
        '''
            # op de oponente
            oponentes_fuertes1 = est_usuario1['stat']['bestWins']['results'][:3]
            ids_oponentes1 = [oponente['opId']['id']
                              for oponente in oponentes_fuertes1]
            for k, id_ops_de_ops in enumerate(ids_oponentes1):
                try:
                    est_usuario2 = obtener_estadisticas(id_ops_de_ops)
                except:
                    continue
                usuarios_col.insert_one({id_ops_de_ops: ''})
                usuarios_col.update_one({id_ops_de_ops: {'$exists': 'true'}}, {
                                        '$set': est_usuario2})
                instruccion += f'''
        MERGE (u{i}{k}:Usuario {{id:"{id_ops_de_ops}"}})
        MERGE (u{i})-[:gano_a]->(u{i}{k})
        '''

        session.run(instruccion)

        # Union con los equipos
        id_usuario = usuario['id']
        instruccion = f'''
      MERGE (u:Usuario {{id:"{id_usuario}"}})
      MERGE (e:Equipo {{nombre:"{equipo['name']}"}})
      MERGE (u)-[:pertenece]->(e)
      '''
        session.run(instruccion)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

## Consultas 

### Análisis de los mejores jugadores de Blitz por cada equipo

Para esta consulta, haremos un muestro a través de la API con 10 jugadores por cada equipo, esto para no saturar la peticiones que nos ofrece API REST y no saaturar el tiempo de carga en la inserción y creación de lo nodos. Para este enfoque utilizaremos MongoDB para guardar cada JSON devuelto de los jugadores y equipos en diferentes colección para posteriormente extraer la información a través de una consulta.


In [95]:
equipo_top5 = {}

# Obtención de los equipos
query = f'''
MATCH (equipo:Equipo)
RETURN (equipo)
'''.replace('\n', '')
nodos_equipos = session.run(query).data()

# Obtencion de los usuarios de cada equipo
for equipo in nodos_equipos:
    nombre_equipo = equipo['equipo']['nombre']
    query=f'''
    MATCH (e:Equipo {{nombre: "{nombre_equipo}"}})
    MATCH (u:Usuario)-[pe:pertenece]-(e)
    RETURN u, e
    '''
    nodos_miembros = session.run(query).data()

    # Obteniendo el top 5 por equipo
    ratings_miembro = []
    for miembro in nodos_miembros:
        usuario_id = miembro['u']['id']
        usuario = usuarios_col.find_one({f'{usuario_id}.id': f'{usuario_id}'})
        rating_blitz = usuario[usuario_id]['perfs']['blitz']['rating']
        username = usuario[usuario_id]['username']
        ratings_miembro.append((username, rating_blitz))
    top5_blitz = sorted(ratings_miembro, key=lambda x: x[1], reverse=True)[:5]

    equipo_top5[nombre_equipo] = top5_blitz

In [439]:
### Distribución del mejor rating por cada equipo de la UNAM
mejores_equipo = list(map(lambda x: (x,equipo_top5[x][0]), equipo_top5))
mejores_equipo_df = pd.DataFrame(dict(
    nombre_equipo=map(lambda x: x[0], mejores_equipo),
    usuario=map(lambda x: x[1][0], mejores_equipo),
    rating=map(lambda x: x[1][1], mejores_equipo),
))

title='Distribución del mejor rating por cada equipo de la UNAM en blitz'
fig = px.bar(mejores_equipo_df.sort_values('rating', ascending=False), x='nombre_equipo', y='rating',#text_auto=True,
             template='plotly_dark', title=title, text='usuario')
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)             
fig.show()

![adjfk](https://raw.githubusercontent.com/gandres-dev/Bases-datos-no-estructuradas/main/img/consulta1_memo.png)

Podemos observar a partir de esta gráfica que cada equipo esta baleanciado con usuarios con un buen rating en partidas de blitz.

### Análisis de coincidencias de las mejores jugadas en blitz.

Para este próximo análisis queremos conocer si existen coincidencias con judagadores que han vencidos con jugadores del mismo equipo. Es decir, determinar si jugadores del mismo equipo han vencido a mismo jugadores.  Con este análisis logramos determinar si los jugadores que vencen al mismo jugador tienen el mismo rating o que caracteristica deberia de tener.

<!-- ![Ilustracion](https://raw.githubusercontent.com/gandres-dev/Bases-datos-no-estructuradas/main/img/ilustracionConsulta2.png) -->


<img src="https://raw.githubusercontent.com/gandres-dev/Bases-datos-no-estructuradas/main/img/ilustracionConsulta2.png" alt="ilustracion" width=50%>


Ver si los jugadores con los que coinciden, es decir, con los que ganaron, tienen el mismo raiting o son muy dispersos.

Con esta gráfico podemos ver claramente un ejemplo de lo queremos lograr encontrar, buscar esos nodos en común dato un equipo. Para lograrlo generamos una query para encontrar los nodos que tienen el mismo equipo y regresar los nodos y aristas que coincida con la relación "gano\_a" para despues a traves de python determinar las coincidencias. Como podemos en la salidos los jugadores "alejandror0210" y  "alanrv" le han ganado los dos al jugador 'maxmledesma' teniendo el mismo rating.

In [191]:
nombre_equipo = "Club Prometeo"

query = f'''
MATCH (e:Equipo {{nombre: "{nombre_equipo}"}})
MATCH (u:Usuario)-[ga:gano_a]-(u2:Usuario)
MATCH (e)-[pe:pertenece]-(u)
RETURN u, ga, u2
'''.replace('\n', '')
nodos_judadores = session.run(query).data()

relaciones_ganoa = [nodo_jugador['ga'] for nodo_jugador in nodos_judadores]
relaciones_ganoa = [(relacion[0]['id'] ,relacion[2]['id'])for relacion in relaciones_ganoa]
usuarios2 = list(map(lambda x: x[1], relaciones_ganoa))
oponentes_coincidentes = list(filter(lambda x: usuarios2.count(x[1]) > 1, relaciones_ganoa))
oponentes_coincidentes

[('alejandror0210', 'maxmledesma'), ('alanrv', 'maxmledesma')]

In [437]:
print("Nombre\t\t\tRanking")
for usuario_id, u2 in oponentes_coincidentes:        
    tmp = usuarios_col.find_one({f'{usuario_id}.id': f'{usuario_id}'})
    raiting_usuario = tmp[usuario_id]['perfs']['correspondence']['rating']
    print(f"{usuario_id}-{raiting_usuario}")

Nombre			Ranking
alejandror0210-1500
alanrv-1500


### Análisis de  jugadores que han crecido a lo largo del tiempo en Blitz dado equipo.

Se analizará el historial de puntos de cada uno de los jugadores de un equipo, para determinar aquellos que han ido subiendo mejorando a traves del tiempo mediante su puntuación.


In [433]:
def jugadores_creciendo_blitz(username, visualizar=False):  
    """Encuentra jugadores que van creciendo en raiting por intervalo de tiempo"""  
    response = requests.get(f'https://lichess.org/api/user/{username}/rating-history')    
    # En el caso, donde el jugador no tenga ninguna partida por lo que implica
    # que no tiene puntos .
    try:
        historial_blitz = response.json()[1]['points']
    except:
        return None
    historial_blitz_array = np.array(historial_blitz)
    tamanio_hist = historial_blitz_array.shape[0]
    ventana = 5
    # Solo si es mayor al tamaño de la ventana 
    if tamanio_hist < ventana:
        return None

    año=historial_blitz_array[:, 0]
    mes=historial_blitz_array[:, 1]
    dia=historial_blitz_array[:, 2]
    fechas = []
    for año, mes, dia in zip(año, mes, dia):    
        fechas.append(f'{año}/{mes+1}/{dia}') 

    historial_blitz_df = pd.DataFrame(dict(
        fechas=fechas,
        puntos=historial_blitz_array[:, 3]
    ))    

    if visualizar:
        fig = px.line(historial_blitz_df, x='fechas', y="puntos",
                    template='plotly_dark', title='Historial rating de un usuario')
        fig.show()
    
    n = historial_blitz_df.shape[0]    
    
    p2 = historial_blitz_df[-ventana:]['puntos'].values
    p1 = historial_blitz_df[n-(ventana+1):-1]['puntos'].values       
    relacion_raiting = np.sum(p2-p1)
    if relacion_raiting > 0:
        return username
    return None    

In [434]:
nombre_equipo = "Club Prometeo"

query = f'''
MATCH (e:Equipo {{nombre: "{nombre_equipo}"}})
MATCH (e)-[pe:pertenece]-(u)
RETURN u
'''.replace('\n', '')
nodos_judadores = session.run(query).data()

# Obtenemos el historial de un jugador
username=nodos_judadores[0]['u']['id']
jugadores_creciendo_blitz(username, visualizar=True)

![img2](https://raw.githubusercontent.com/gandres-dev/Bases-datos-no-estructuradas/main/img/consulta2_memo.png)

A partir de esta gráfica de ejemplo, podemos analizar las ultimas fechas para determinar si el jugador va creciendo conforme pasan los dias o en picada.

In [384]:
jugadores_creciendo = []
# Obtenemos el historial de todos los jugadores de un equipo
for jugador in nodos_judadores:
    username=jugador['u']['id']
    jugador_creciendo = jugadores_creciendo_blitz(username)
    if jugador_creciendo:
        jugadores_creciendo.append(jugador_creciendo)

In [436]:
print("******Jugadores que van creciendo en las ultimas 5 fechas*****")
output=list(map(print, jugadores_creciendo))

******Jugadores que van creciendo en las ultimas 5 fechas*****
xschoko
xngelguale
alanmezal


In [435]:
username=nodos_judadores[0]['u']['id']
jugadores_creciendo_blitz('xschoko', visualizar=True)

'xschoko'

![sdlk](https://raw.githubusercontent.com/gandres-dev/Bases-datos-no-estructuradas/main/img/consulta3_memo.png)

Con esta gráfica podemos ver que precisamente el jugador `xschoko` va creciendo su rating en ultimas 5 fechas.